In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('vaccine_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...


In [ ]:
def get_keyword_occurrence_time_series(query_keywords, cond='or'):
    
    # (2) proceed filtering: news category
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = df.contents.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    else:
        # query keywords condition使用者輸入關鍵字條件
        condition = df.contents.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()

    # condiction is a list of True or False boolean value
    df_query = df[condition]

    query_freq = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'freq':[1 for _ in range(len(df_query))]})


    # 開始時間、結束時間兩項必須也加入到query_freq，計算次數時才會有完整的時間軸，否則時間軸長度會因為新聞時間不同，導致時間軸不一致
    dt_start_date = datetime.strptime(start_date, '%Y-%m-%d')
    dt_end_date = datetime.strptime(end_date, '%Y-%m-%d')    
    query_freq = query_freq.append({'date_index': dt_start_date, 'freq': 0}, ignore_index=True)
    query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)

    freq_data = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()

    freq_data.reset_index(inplace=True)

    # 只有y, 沒有時間變數x
    y_freq_data = freq_data.freq.to_list()
    # 有時間變數x,y
    line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_data.date_index,freq_data.freq)]

    return line_xy_data, y_freq_data
